In [1]:
SEQ_LEN = 30

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split

In [2]:
from transformers import BertTokenizer
from transformers import AutoTokenizer
import tensorflow as tf

2024-04-16 17:45:41.778697: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 17:45:41.778804: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 17:45:41.908127: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
df = pd.read_csv('https://archive.org/download/fine-tune-bert-tensorflow-train.csv/train.csv.zip', 
                 compression = 'zip', low_memory = False)
df.shape

(1306122, 3)

In [8]:
df.tail(20)

,qid,question_text,target
1306102,ffff3778790af9baae76,What steps can I take to live a normal life if...,0
1306103,ffff3f0a2449ffe4b9ff,Isn't Trump right after all? Why should the US...,1
1306104,ffff41393389d4206066,Is 33 too late for a career in creative advert...,0
1306105,ffff42493fc203cd9532,What is difference between the filteration wor...,0
1306106,ffff48dd47bee89fff79,"If the universe ""popped"" into existence from n...",0
1306107,ffff5fd051a032f32a39,How does a shared service technology team meas...,0
1306108,ffff6d528040d3888b93,How is DSATM civil engineering?,0
1306109,ffff8776cd30cdc8d7f8,Do you know any problem that depends solely on...,0
1306110,ffff94d427ade3716cd1,What are some comic ideas for you Tube videos ...,0
1306111,ffffa382c58368071dc9,"If you had $10 million of Bitcoin, could you s...",0


In [11]:
train_df, remaining = train_test_split(df, random_state = 42, train_size = 0.0075, stratify = df['target'])
val_df, _ = train_test_split(remaining, random_state=42, train_size = 0.00075, stratify = remaining['target'])

print(train_df.shape, val_df.shape)

(9795, 3) (972, 3)


In [13]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []

    for t in text.split(" "):
        t = '' if t.startswith('@') and len(t) > 1 else t
        t = '' if t.startswith('http') else t
        t = t.replace("#","")
        new_text.append(t.lower())
    return " ".join(new_text).strip().replace("  ", " ")

In [16]:
train_df['question_text'] = train_df['question_text'].apply(preprocess)
train_df.head()

,qid,question_text,target
24766,04dad2b5f2eb9d7b9584,why are unhealthy relationships so desirable?,0
1184991,e8389c8a9fc7db099491,which war changed the course of history of the...,0
1283032,fb73e2203cff654b57de,i have started a youtube channel named askhamy...,0
81937,100ba1e81f80494ed600,how did steve nash make the nba if he started ...,0
411225,5094ac3d788015df2f2b,what are the health benefits of eating clean?,0


In [17]:
# example of tokenization
tokenizer.tokenize(preprocess("Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all"))

['our',
 'deeds',
 'are',
 'the',
 'reason',
 'of',
 'this',
 'earthquake',
 'may',
 'allah',
 'forgive',
 'us',
 'all']

In [18]:
# example of an output from encoded text by tokeneizer
encoded_input = tokenizer(preprocess("Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all"))
print(encoded_input)

{'input_ids': [101, 2256, 15616, 2024, 1996, 3114, 1997, 2023, 8372, 2089, 16455, 9641, 2149, 2035, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [19]:
tokenizer.decode(encoded_input['input_ids'])

'[CLS] our deeds are the reason of this earthquake may allah forgive us all [SEP]'

In [20]:
with tf.device('/cpu:0'):
    train_data = tf.data.Dataset.from_tensor_slices((train_df['question_text'].values, train_df['target'].values))
    val_data = tf.data.Dataset.from_tensor_slices((val_df['question_text'].values, val_df['target'].values))
    for text, label in train_data.take(1):
        print(text)
        print(label)

tf.Tensor(b'why are unhealthy relationships so desirable?', shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)


In [24]:
# encoding the training data
train_encoded_inputs = tokenizer(train_df['question_text'].tolist(),
                                 add_special_tokens = True,
                                 padding='max_length', 
                                 truncation=True, 
                                 max_length=SEQ_LEN, 
                                 return_token_type_ids=False,
                                 return_tensors = 'tf')

In [25]:
train_encoded_inputs

{'input_ids': <tf.Tensor: shape=(9795, 30), dtype=int32, numpy=
array([[ 101, 2339, 2024, ...,    0,    0,    0],
       [ 101, 2029, 2162, ...,    0,    0,    0],
       [ 101, 1045, 2031, ..., 2000, 2265,  102],
       ...,
       [ 101, 2129, 2079, ...,    0,    0,    0],
       [ 101, 2129, 2116, ...,    0,    0,    0],
       [ 101, 2339, 2106, ...,    0,    0,    0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(9795, 30), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [26]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_encoded_inputs, train_df['target'].values))

In [27]:
# formatting the data as required by bert model
def map_bert(inputs, labels):
    inputs = {'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask']}
    return inputs, labels

In [28]:

train_dataset = train_dataset.map(map_bert)

In [29]:
for t in train_dataset.take(2):
    print (t)

({'input_ids': <tf.Tensor: shape=(30,), dtype=int32, numpy=
array([  101,  2339,  2024,  4895, 20192, 24658,  2100,  6550,  2061,
       16166,  1029,   102,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(30,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)>}, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
({'input_ids': <tf.Tensor: shape=(30,), dtype=int32, numpy=
array([ 101, 2029, 2162, 2904, 1996, 2607, 1997, 2381, 1997, 1996, 2088,
       1029,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(30,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [30]:
dataset = train_dataset.shuffle(100000).batch(64)

In [31]:
# length of the dataset(total batches)
DS_LEN = len(dataset)
DS_LEN

154

In [32]:
# take 80% for train and 20% for validation
SPLIT = 0.8
train_ds = dataset.take(round(DS_LEN*SPLIT))
val_ds = dataset.skip(round(DS_LEN*SPLIT))

In [33]:
from transformers import TFAutoModel

In [34]:
bert = TFAutoModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [41]:
# create model architecture
#n_classes = len((train_df.target.unique()))

# Input layers
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN,), dtype=np.int32, name='input_ids' )
mask = tf.keras.layers.Input(shape=(SEQ_LEN,), dtype=np.int32, name = 'attention_mask')

# bert embeddings
print(bert([input_ids, mask])[0])
embeddings = bert([input_ids, mask])[0]
print(embeddings,"embeddings")
cls_token = embeddings[:,0,:]

# keras layers
#x = tf.keras.layers.GlobalMaxPool1D()(embeddings)
x = tf.keras.layers.BatchNormalization()(cls_token)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)

# output layer
y = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# create the model
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

KerasTensor(type_spec=TensorSpec(shape=(None, 30, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0', description="created by layer 'tf_bert_model'")
KerasTensor(type_spec=TensorSpec(shape=(None, 30, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0', description="created by layer 'tf_bert_model'") embeddings


In [43]:
bert([input_ids, mask])[1]

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>

In [36]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 30)]                 0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 30)]                 0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, 30, 76                                       

In [38]:
import transformers

In [37]:
#learning_rate = 1e-3

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)
#optimizer = transformers.AdamWeightDecay(learning_rate=5e-4)
loss = tf.keras.losses.BinaryCrossentropy()
metric = tf.keras.metrics.BinaryAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [39]:
# train the model
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.1,min_delta=0.001,monitor='val_loss'),
             tf.keras.callbacks.EarlyStopping(patience=5, min_delta=0.001, monitor='val_loss')]

history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 50,
    callbacks = callbacks
)

Epoch 1/50


I0000 00:00:1713290278.625134      86 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


123/123 [==============================] - 163s 510ms/step - loss: 0.2608 - accuracy: 0.9267 - val_loss: 0.3314 - val_accuracy: 0.9371 - lr: 5.0000e-04
Epoch 2/50
123/123 [==============================] - 51s 412ms/step - loss: 0.2529 - accuracy: 0.9372 - val_loss: 0.2413 - val_accuracy: 0.9449 - lr: 5.0000e-04
Epoch 3/50
123/123 [==============================] - 50s 407ms/step - loss: 0.2359 - accuracy: 0.9409 - val_loss: 0.2437 - val_accuracy: 0.9350 - lr: 5.0000e-04
Epoch 4/50
123/123 [==============================] - 50s 409ms/step - loss: 0.2483 - accuracy: 0.9375 - val_loss: 0.2830 - val_accuracy: 0.9303 - lr: 5.0000e-04
Epoch 5/50
123/123 [==============================] - 50s 408ms/step - loss: 0.2421 - accuracy: 0.9386 - val_loss: 0.3888 - val_accuracy: 0.9423 - lr: 5.0000e-05
Epoch 6/50
123/123 [==============================] - 50s 408ms/step - loss: 0.2422 - accuracy: 0.9388 - val_loss: 0.3728 - val_accuracy: 0.9438 - lr: 5.0000e-05
Epoch 7/50
123/123 [==================